In [1]:
import os
from pathlib import Path

# 获取当前工作目录
notebook_dir = Path(os.getcwd())

print(f"Notebook directory: {notebook_dir}")


Notebook directory: /home/ye/code/DarwinKit


In [2]:
import torch
import torch.nn as nn
from dataclasses import dataclass
from pathlib import Path



@dataclass
class MyModelConfig:
    input_size:int = 12
    hidden_size:int = 12
    output_size:int = 12

class MyModel(nn.Module):
    def __init__(self, config:MyModelConfig):
        super(MyModel, self).__init__()
        self.config = config
        self.fc1 = nn.Linear(config.input_size, config.hidden_size)
        self.fc2 = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from darkit.core import Trainer, TrainerConfig

from typing import Optional

model_cache_path = notebook_dir / '.cache'

if not model_cache_path.exists():
    model_cache_path.mkdir()

print(f"model_cache_path: {model_cache_path}")

@dataclass
class MyTrainerConfig(TrainerConfig):
    device = "cuda"
    lr = 1e-3
    batch_size = 4
    max_step: int = 10000 # 定义最大训练步数
    save_step_interval: int = 10000 # 定义保存模型间隔


class MyTrainer(Trainer):
    def __init__(self, model:MyModel, config:MyTrainerConfig, **kwargs):
        super().__init__(model, config, **kwargs)
        self.config = config
    
    @property
    def root(self):
        return model_cache_path

    def _get_optimizer(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.config.lr)
    
    def train(self, train_dataset, val_dataset=None):
        # 在这里实现自定义的训练逻辑
        dataloader = DataLoader(train_dataset, batch_size=self.config.batch_size)
        
        self.optimizer = self._get_optimizer()
        for step, batch in tqdm(enumerate(dataloader)):
            inputs, labels = batch
            inputs = inputs.to(self.config.device)
            labels = labels.to(self.config.device)

            outputs = self.model(inputs)
            loss = F.mse_loss(outputs, labels)
            loss.backward()
            self.optimizer.step()
            
            # 调用 Trainer 父类提供的保存和评估方法
            # 会根据 config 提供的参数来控制保存和评估的逻辑
            self.current_step = step
            self._auto_save_pretrained()

Trainer.register(MyModel.__name__, MyTrainer)

name = f"TEST_MODEL_{random.randint(10,99)}"
mconf = MyModelConfig()
model = MyModel(mconf)
tconf = MyTrainerConfig(name=name)
trainer = Trainer(model, tconf)

# 生成一些随机数据
x = torch.randn(12)
y = torch.randn(12)
train_dataset = [(torch.tensor(x), torch.tensor(y)) for _ in range(4*10000)]

trainer.train(train_dataset)

/home/ye/miniconda3/envs/darkit/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model_cache_path: /home/ye/code/DarwinKit/.cache
{'name': 'TEST_MODEL_43', 'max_step': 10000, 'save_step_interval': 10000}
Cannot retrieve source code for built-in class: <class '__main__.MyModel'> is a built-in class
Logger initialized at /home/ye/code/DarwinKit/.cache/TEST_MODEL_43/train_log.csv


/tmp/ipykernel_48339/535331876.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = [(torch.tensor(x), torch.tensor(y)) for _ in range(4*10000)]
10000it [00:12, 774.74it/s]

Model saved at /home/ye/code/DarwinKit/.cache/TEST_MODEL_43
Model saved epoch 10000/10000 at iter-10000-ckpt


In [4]:
from darkit.core import Predicter

class MyPredicter(Predicter):
    def __init__(self, name, model, device="cpu"):
        super().__init__(name, model, device)
    
    @classmethod
    def get_root(cls) -> Path:
        return model_cache_path
    
    @classmethod
    def get_model(cls, name:str, checkpoint: Optional[str] = None):
        # 在这里实现自定义的获取模型的逻辑
        checkpoint_path = cls.get_checkpoint(name, checkpoint)
        config_dict = cls.get_model_config_json(name)
        config = MyModelConfig(**config_dict)
        model = MyModel(config=config)
        checkpoint_dict = torch.load(checkpoint_path, weights_only=True)
        model.load_state_dict(checkpoint_dict["state_dict"], strict=True)
        return model
    
    def predict(self, input):
        input = input.to(self.device)
        output = self.model(input)
        return output
    
MyPredicter.register(MyModel.__name__, MyPredicter)


predicter = MyPredicter.from_pretrained(name)
predicter.predict(torch.randn(12))

tensor([-145.0064,   68.5688,  502.8868, -548.7836,  -20.6441,  131.5918,
        -194.9134,  166.2308,  327.5081,  171.0134, -473.6718,   25.5564],
       device='cuda:0', grad_fn=<ViewBackward0>)